In [1]:
!pip install google-generativeai pypdf requests transformers torch accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.1 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [2]:
import google.generativeai as genai
from pypdf import PdfReader
import requests
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os
import bitsandbytes

tokenizer = None
model = None
RFP_text_files = []
RFP_response_text_files = []
LLM_verdict = [] # This remains global as results are appended to it

In [3]:
"""
import zipfile
import os

def unzip_files(dataset_folder):
    for filename in os.listdir(dataset_folder):
        if filename.endswith(".zip"):
            filepath = os.path.join(dataset_folder, filename)
            try:
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(dataset_folder)
                print(f"Successfully unzipped {filename}")
            except zipfile.BadZipFile:
                print(f"Error: {filename} is not a valid zip file")
            except Exception as e:
                print(f"An error occurred while unzipping {filename}: {e}")

# Replace 'dataset' with the actual path to your dataset folder if it's different
dataset_folder_path = '/kaggle/input/dataset-tln/dataset'
unzip_files(dataset_folder_path)
"""

'\nimport zipfile\nimport os\n\ndef unzip_files(dataset_folder):\n    for filename in os.listdir(dataset_folder):\n        if filename.endswith(".zip"):\n            filepath = os.path.join(dataset_folder, filename)\n            try:\n                with zipfile.ZipFile(filepath, \'r\') as zip_ref:\n                    zip_ref.extractall(dataset_folder)\n                print(f"Successfully unzipped {filename}")\n            except zipfile.BadZipFile:\n                print(f"Error: {filename} is not a valid zip file")\n            except Exception as e:\n                print(f"An error occurred while unzipping {filename}: {e}")\n\n# Replace \'dataset\' with the actual path to your dataset folder if it\'s different\ndataset_folder_path = \'/kaggle/input/dataset-tln/dataset\'\nunzip_files(dataset_folder_path)\n'

In [4]:
RFP_PDF_files = []
RFP_response_PDF_files = []
for i in range(1,15):
  RFP_PDF_files.append(f'/kaggle/input/dataset-tln/dataset/{i}_(DCE and Answer)/{i}_CCTP.pdf')
  RFP_response_PDF_files.append(f'/kaggle/input/dataset-tln/dataset/{i}_(DCE and Answer)/{i}_Response.pdf')

In [5]:
RFP_text_files = []
RFP_response_text_files = []

In [6]:
def read_RFP_file(PDF_file):
  text = ""
  reader = PdfReader(PDF_file)
  for page in reader.pages:
     text+= page.extract_text()
  return text

for PDF_file in RFP_PDF_files :
  RFP_text_files.append(read_RFP_file(PDF_file))
for PDF_file in RFP_response_PDF_files :
  RFP_response_text_files.append(read_RFP_file(PDF_file))

In [7]:
def get_prompt_template(rfp_text, response_text) :
  return f"""Vous allez évaluer la qualité des appels d'offres (RFP) et des réponses aux appels d'offres.
    Donnez directement une note de cohérence de 0 à 10, 0 étant la pire et 10 étant la meilleure.
    Si la note est supérieure ou égale à 5, indiquez la probabilité que la réponse au RFP soit acceptée (0 signifiant qu'elle ne sera pas acceptée, 1 signifiant qu'elle le sera).
    La note et la probabilité (le cas échéant) doivent être les dernières informations de votre réponse. Ces deux métriques sont importantes et doivent toujours être fournies.
    Vos réponses doivent être en français, car nos clients sont tous français.
    Donne la réponse la plus minimale possible
    RFP: {rfp_text} 
    Réponse: {response_text}
    Évaluation:"""

In [30]:
def generate_ideas_batch(rfp_text, response_text):
    global tokenizer, model

    if tokenizer is None or model is None:
        print("Warning: Tokenizer and/or model not loaded. Returning empty list.")
        return []

    print("Starting memory-safe generation loop...")

    model_context_window = getattr(model.config, 'max_position_embeddings', tokenizer.model_max_length)
    if model_context_window > 1e10:
        model_context_window = 4096

    sample_prompt = get_prompt_template("", "")
    prompt_tokens = len(tokenizer.encode(sample_prompt))

    reserved_for_output = 500
    max_input_tokens = model_context_window - prompt_tokens - reserved_for_output - 50
    max_doc_tokens = max(0, max_input_tokens // 2)

    
    final_responses = ""


    # Truncate and format prompt
    rfp_tokens = tokenizer.encode(rfp_text)[:max_doc_tokens]
    response_tokens = tokenizer.encode(response_text)[:max_doc_tokens]

    rfp_text = tokenizer.decode(rfp_tokens, skip_special_tokens=True)
    response_text = tokenizer.decode(response_tokens, skip_special_tokens=True)
    prompt = get_prompt_template(rfp_text, response_text)

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=model_context_window)

            # Move input to appropriate device (try CPU if GPU is full)
        try:
                device = model.device if torch.cuda.is_available() else torch.device("cpu")
                inputs = {k: v.to(device) for k, v in inputs.items()}
                model.to(device)
        except Exception:
                print("Falling back to CPU due to device transfer failure.")
                device = torch.device("cpu")
                model.to(device)
                inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
                output = model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    temperature=0.7,
                    max_new_tokens=reserved_for_output,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id,
                )

        generated = output[0][inputs["input_ids"].shape[1]:]
        decoded = tokenizer.decode(generated, skip_special_tokens=True).strip()
        final_responses += decoded

        print(f"✅ Generated response {len(final_responses)} ")

    except Exception as e:
        print(f"⚠️ Error on pair {idx}: {e}")
        import traceback
        traceback.print_exc()
        

    return final_responses


In [31]:
def generate_ideas(i):
    """
    Processes a single RFP and response pair.
    This function acts as a wrapper to call the batch processing function
    for a single item.
    """
    global RFP_text_files, RFP_response_text_files, LLM_verdict

    # Extract the single RFP and response for this iteration
    single_rfp_text = RFP_text_files[i]
    single_response_text = RFP_response_text_files[i]

    # Call the batch processing function with a list containing only the current item
    # This maintains the batch processing logic even for a single item.
    results = generate_ideas_batch(single_rfp_text, single_response_text)
    
    # Append the result(s) to the global LLM_verdict list
    LLM_verdict.extend(results) # Use extend as generate_ideas_batch returns a list

def main():
    """
    Main function for processing RFP files, adapted to the requested loop structure.
    """
    print("Starting main processing loop...")
    for i in range(len(RFP_text_files)):
        print(f"Processing RFP pair {i+1}/{len(RFP_text_files)}")
        generate_ideas(i)
    
    if LLM_verdict:
        print("\n--- Example output (first verdict) ---")
        print(LLM_verdict[0])
        print("\n--- All verdicts generated ---")
        # You might want to print all verdicts for review, or save them to a file
        # for verdict_idx, verdict_text in enumerate(LLM_verdict):
        #     print(f"Verdict {verdict_idx}: {verdict_text}\n---")
    else:
        print("No responses generated.")

In [32]:
"""!zip -r /content/dataset.zip /content/dataset
files.download("/content/dataset.zip")"""


'!zip -r /content/dataset.zip /content/dataset\nfiles.download("/content/dataset.zip")'

In [33]:
#!pip install -U bitsandbytes

In [34]:
if __name__ == "__main__":
    # --- Model and Tokenizer Loading ---
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # Recommended choice

    try:
        print(f"Attempting to load model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="auto", 
            trust_remote_code=False 
        )
        model.eval() 

        if tokenizer.pad_token is None:
            if tokenizer.eos_token:
                tokenizer.pad_token = tokenizer.eos_token
            elif tokenizer.unk_token:
                tokenizer.pad_token = tokenizer.unk_token
            else:
                tokenizer.add_special_tokens({'pad_token': '[PAD]'})
                print("Warning: Added a new [PAD] token to tokenizer.")
        
        if tokenizer.eos_token_id is None:
            if tokenizer.eos_token:
                tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
            elif "<|endoftext|>" in tokenizer.vocab:
                tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids("<|endoftext|>")
            elif "<|im_end|>" in tokenizer.vocab:
                 tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids("<|im_end|>")
            else:
                print("Warning: Could not determine eos_token_id. Generation might be affected.")
        
        print(f"Model '{model_id}' loaded successfully with quantization.")
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        if tokenizer.eos_token_id is None:
            if "llama" in model_id.lower() or "mistral" in model_id.lower():
                tokenizer.eos_token_id = tokenizer.unk_token_id 
            else:
                tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids("<|endoftext|>") 
    except Exception as e : 
        print(e)

    main()

Attempting to load model: TinyLlama/TinyLlama-1.1B-Chat-v1.0


Token indices sequence length is longer than the specified maximum sequence length for this model (8243 > 2048). Running this sequence through the model will result in indexing errors


Model 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' loaded successfully with quantization.
Starting main processing loop...
Processing RFP pair 1/14
Starting memory-safe generation loop...
✅ Generated response 1539 
Processing RFP pair 2/14
Starting memory-safe generation loop...
✅ Generated response 1207 
Processing RFP pair 3/14
Starting memory-safe generation loop...
✅ Generated response 1091 
Processing RFP pair 4/14
Starting memory-safe generation loop...
✅ Generated response 1554 
Processing RFP pair 5/14
Starting memory-safe generation loop...
✅ Generated response 1355 
Processing RFP pair 6/14
Starting memory-safe generation loop...
✅ Generated response 1461 
Processing RFP pair 7/14
Starting memory-safe generation loop...
✅ Generated response 1139 
Processing RFP pair 8/14
Starting memory-safe generation loop...
✅ Generated response 38 
Processing RFP pair 9/14
Starting memory-safe generation loop...
✅ Generated response 1153 
Processing RFP pair 10/14
Starting memory-safe generation l

In [35]:
# Save all verdicts to a file
with open("llm_verdicts.txt", "w", encoding="utf-8") as f:
    for i, verdict in enumerate(LLM_verdict, 1):
        f.write(f"--- Verdict {i} ---\n")
        f.write(verdict + "\n\n")
        
        print("All responses saved to llm_verdicts.txt.")


All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses saved to llm_verdicts.txt.
All responses sa

In [ ]:
print(len(RFP_PDF_files))